In [2]:
#%pip install "leafmap[maplibre]" scikit-learn

In [3]:
import os
import pandas as pd
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from leafmap.common import evaluate_model, plot_actual_vs_predicted, download_file

Download the Zillow home value data at the county level.

In [10]:
zhvi_url = "https://github.com/opengeos/datasets/releases/download/us/zillow_home_value_index_by_zipcode.csv"
# Save to home directory to avoid read-only file system issues
zhvi_file = os.path.expanduser("/Users/haritshah/Documents/GitHub/GeoSpatialProjects/PredictingHomeValuesIn3D/Zillow/Zillow Home Value Index by Zipcode.csv")

In [ ]:
# if not os.path.exists(zhvi_file):
#     download_file(zhvi_url, zhvi_file)

Process Zillow Data

In [11]:
zhvi_df = pd.read_csv(zhvi_file, dtype={"RegionName": "string"})
zhvi_df.index = zhvi_df["RegionName"].apply(lambda x: f"zip/{x}")
zhvi_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31
RegionName,,,,,,,,,,,,,,,,,,,,,
zip/77494,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,209050.47676,...,493768.79581,495075.41384,495380.40257,495002.98333,495408.56194,496096.30546,497215.16189,497931.31997,498348.02484,498770.07052
zip/8701,61148,2,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,129618.78060,...,567181.88952,574111.68852,579228.58585,583825.79895,588765.87771,594583.48573,599788.27507,603106.03147,605075.74933,605701.71314
zip/77449,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,103655.52834,...,281613.08126,282147.02210,282107.28078,281872.89042,281691.18638,281502.99185,281086.41028,280298.90222,279570.22174,278960.08768
zip/11368,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,146323.41156,...,448237.17350,452805.95427,453500.45273,452997.37976,452371.99866,453320.49579,453075.34378,451863.57197,449627.30018,447532.07593
zip/77084,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,102106.86183,...,274112.51755,274613.19668,274446.91817,274155.37019,273883.45917,273631.25456,273125.76564,272528.81542,272190.92108,271949.29478


Load PDFM Embeddings data

In [12]:
embeddings_file = os.path.expanduser("/Users/haritshah/Desktop/pdfm_embeddings/v0/us/zcta_embeddings.csv")



In [13]:
zipcode_embeddings = pd.read_csv(embeddings_file).set_index("place")
zipcode_embeddings.head()

,state,county,city,population,latitude,longitude,feature0,feature1,feature2,feature3,...,feature320,feature321,feature322,feature323,feature324,feature325,feature326,feature327,feature328,feature329
place,,,,,,,,,,,,,,,,,,,,,
zip/97910,OR,Malheur County,Jordan Valley,609,42.749076,-117.511459,-0.138227,1.120377,0.072900,0.297442,...,-0.158891,-0.168708,1.231994,-0.155765,3.043214,-0.169749,0.177463,-0.001661,-0.001010,4.495589
zip/89412,NV,Washoe County,Gerlach,98,41.102934,-119.695361,-0.141379,1.422782,0.234269,0.159156,...,-0.157417,-0.043606,2.788701,-0.062547,3.700745,-0.169827,-0.137990,-0.024385,-0.000295,3.399393
zip/88030,NM,Luna County,Deming,24139,32.191634,-107.729431,-0.046666,1.414424,0.146803,1.113256,...,-0.000654,0.437475,4.229295,0.229199,2.098469,1.150497,0.716122,-0.116499,-0.051163,3.866543
zip/82633,WY,Converse County,Douglas,9478,43.022270,-105.410250,-0.090293,1.266280,0.447868,0.781861,...,-0.033771,0.579775,2.688665,0.175669,0.990921,1.644879,0.222517,-0.047864,-0.000042,7.453567
zip/59538,MT,Phillips County,Malta,2936,48.112019,-107.845520,-0.092886,1.256203,-0.050897,0.321954,...,-0.169915,-0.088829,0.338914,-0.102962,-0.156583,1.493696,2.259007,-0.161916,-0.001087,0.972243


Join Zillow and PDFM Data

In [14]:
data = zhvi_df.join(zipcode_embeddings, how="inner")
data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,feature320,feature321,feature322,feature323,feature324,feature325,feature326,feature327,feature328,feature329
zip/77494,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,209050.47676,...,-0.000001,-0.000573,-7.384640e-02,3.741291,6.835684,7.062005e+00,2.521905,1.494393,-0.000000e+00,0.663955
zip/77449,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,103655.52834,...,-0.017497,-0.003990,-1.646668e-01,4.616399,6.331781,6.382573e+00,2.943955,1.990187,-4.000000e-06,1.199221
zip/11368,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,146323.41156,...,-0.099274,2.189546,-2.500000e-07,6.165315,-0.082513,-1.300000e-07,3.876507,-0.031210,-4.438200e-04,3.084769
zip/77084,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,102106.86183,...,-0.006604,-0.015875,-1.620866e-01,4.832126,6.007081,6.243350e+00,1.670585,2.623777,-7.000000e-08,0.925346
zip/79936,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,88633.20026,...,2.817620,3.308279,1.424846e+00,5.052125,-0.169852,-1.595640e-03,0.101079,-0.168736,-4.000000e-08,11.867829


In [15]:
embedding_features = [f"feature{x}" for x in range(330)]
label = "2024-10-31"

In [16]:
data = data.dropna(subset=[label])

Split Train and Test Data

In [17]:
data = data[embedding_features + [label]]
X = data[embedding_features]
y = data[label]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Fit Linear Regression Model

In [18]:
# Initialize and train a simple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

Evaluate Linear Regression Model

In [19]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
metrics = evaluate_model(evaluation_df)
print(metrics)

{'r2': 0.7923863392337245, 'r': 0.8908469152635229, 'rmse': 131888.3874333895, 'mae': 72573.83922119638, 'mape': 0.26673190601505936}


In [20]:
xy_lim = (0, 3_000_000)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted Home Values",
    x_label="Actual Home Value",
    y_label="Predicted Home Value",
)

Fit K-Nearest Neighbors Model

In [22]:
k = 5
model = KNeighborsRegressor(n_neighbors=k)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

Evaluate K-Nearest Neighbors Model

In [23]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
# Evaluate the model
metrics = evaluate_model(evaluation_df)
print(metrics)

{'r2': 0.787670314194312, 'r': 0.8889091262880303, 'rmse': 133377.92408499756, 'mae': 60408.591463699224, 'mape': 0.2067017807110204}


In [24]:
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted Home Values",
    x_label="Actual Home Value",
    y_label="Predicted Home Value",
)